![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

The Head Data Scientist at *Training Data Ltd*. has asked you to create a DataFrame called `ds_jobs_clean` that stores the data in `customer_train.csv` much more efficiently. Specifically, they have set the following requirements:

* Columns containing integers must be stored as 32-bit integers (`int32`).
* Columns containing floats must be stored as 16-bit floats (`float16`).
* Columns containing nominal categorical data must be stored as the `category` data type.
* Columns containing ordinal categorical data must be stored as *ordered categories*, and not mapped to numerical values, with an order that reflects the natural order of the column.
* The columns of `ds_jobs_clean` must be in the same order as the original dataset.
* The DataFrame should be filtered to only contain students with *10 or more years* of experience at companies with *at least 1000 employees*, as their recruiter base is suited to more experienced professionals at enterprise companies.

If you call `.info()` or `.memory_usage()` methods on `ds_jobs` and `ds_jobs_clean` after you've preprocessed it, you should notice a substantial decrease in memory usage.

In [78]:
# Start your code here!
import pandas as pd

dtypes = {'student_id': 'int32', 'city': 'category', 'city_development_index': 'float16',
          'gender': 'category', 'relevant_experience': 'category', 'enrolled_university': 'category',
          'education_level': 'category', 'major_discipline': 'category', 'experience': 'category',
          'company_size': 'category', 'company_type': 'category', 'last_new_job': 'category',
          'training_hours': 'int32', 'job_change': 'int32'}

In [79]:
ds_jobs = pd.read_csv('customer_train.csv')
ds_jobs_clean = pd.read_csv('customer_train.csv', dtype=dtypes)
ds_jobs_clean.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.919922,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.775879,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624023,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789062,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767090,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0


In [80]:
ds_jobs_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  category
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  int32   
dtypes: category(10), float

In [81]:
ds_jobs['city'].describe()

count        19158
unique         123
top       city_103
freq          4355
Name: city, dtype: object

In [82]:
ds_jobs['gender'].value_counts(dropna=False)

gender
Male      13221
NaN        4508
Female     1238
Other       191
Name: count, dtype: int64

In [83]:
ds_jobs['relevant_experience'].value_counts(dropna=False)

relevant_experience
Has relevant experience    13792
No relevant experience      5366
Name: count, dtype: int64

In [84]:
relevant_experience_order = ['No relevent experience', 'Has relevent experience']
ds_jobs_clean['relevant_experience'] = pd.Categorical(ds_jobs_clean['relevant_experience'], categories=relevant_experience_order, ordered=True)

In [85]:
ds_jobs['enrolled_university'].value_counts(dropna=False)

enrolled_university
no_enrollment       13817
Full time course     3757
Part time course     1198
NaN                   386
Name: count, dtype: int64

In [86]:
enrolled_university_order = ['no_enrollment', 'Part time course', 'Full time course']
ds_jobs_clean['enrolled_university'] = pd.Categorical(ds_jobs_clean['enrolled_university'], categories=enrolled_university_order, ordered=True)

In [87]:
ds_jobs['education_level'].value_counts(dropna=False)

education_level
Graduate          11598
Masters            4361
High School        2017
NaN                 460
Phd                 414
Primary School      308
Name: count, dtype: int64

In [88]:
education_level_order = ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd']
ds_jobs_clean['education_level'] = ds_jobs_clean['education_level'].cat.reorder_categories(education_level_order, ordered=True)

In [89]:
ds_jobs['major_discipline'].value_counts(dropna=False)

major_discipline
STEM               14492
NaN                 2813
Humanities           669
Other                381
Business Degree      327
Arts                 253
No Major             223
Name: count, dtype: int64

In [90]:
ds_jobs['experience'].value_counts(dropna=False)

experience
>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
15      686
11      664
14      586
1       549
<1      522
16      508
12      494
13      399
17      342
19      304
18      280
20      148
NaN      65
Name: count, dtype: int64

In [91]:
experience_order = ['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20']
ds_jobs_clean['experience'] = ds_jobs_clean['experience'].cat.reorder_categories(experience_order, ordered=True)
ds_jobs_clean = ds_jobs_clean[ds_jobs_clean['experience'] >= '10']

In [92]:
ds_jobs['company_size'].value_counts(dropna=False)

company_size
NaN          5938
50-99        3083
100-499      2571
10000+       2019
10-49        1471
1000-4999    1328
<10          1308
500-999       877
5000-9999     563
Name: count, dtype: int64

In [93]:
company_size_order = ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+']
ds_jobs_clean['company_size'] = ds_jobs_clean['company_size'].cat.reorder_categories(company_size_order, ordered=True)
ds_jobs_clean = ds_jobs_clean[ds_jobs_clean['company_size'] >= '1000-4999']

In [94]:
ds_jobs['company_type'].value_counts(dropna=False)

company_type
Pvt Ltd                9817
NaN                    6140
Funded Startup         1001
Public Sector           955
Early Stage Startup     603
NGO                     521
Other                   121
Name: count, dtype: int64

In [95]:
ds_jobs['last_new_job'].value_counts(dropna=False)

last_new_job
1        8040
>4       3290
2        2900
never    2452
4        1029
3        1024
NaN       423
Name: count, dtype: int64

In [96]:
last_new_job_order = ['never', '1', '2', '3', '4', '>4']
ds_jobs_clean['last_new_job'] = ds_jobs_clean['last_new_job'].cat.reorder_categories(last_new_job_order, ordered=True)